In [1]:
import pandas as pd
import numpy as np
from IPython.display import display, Markdown
import re

# 2. Traitements

Pour commencer, nous importons la base de données.

In [2]:
FoodData = pd.read_feather('data/FoodData_filtered.feather')

In [3]:
FoodData.head()

,code,url,last_modified_datetime,image_small_url,product_name,quantity,pnns_groups_1,pnns_groups_2,food_groups,nutriscore_grade,energy-kcal_100g,fat_100g,saturated-fat_100g,carbohydrates_100g,sugars_100g,proteins_100g,salt_100g,sodium_100g
0,0000000000100,http://world-fr.openfoodfacts.org/produit/0000...,2015-10-12 14:13:32+00:00,https://images.openfoodfacts.org/images/produc...,moutarde au moût de raisin,100g,Fat and sauces,Dressings and sauces,en:dressings-and-sauces,d,223.559759,8.2,2.2,29.0,22.0,5.1,4.600,1.840
1,0000000000949,http://world-fr.openfoodfacts.org/produit/0000...,2019-08-08 12:46:52+00:00,https://images.openfoodfacts.org/images/produc...,Salade de carottes râpées,None,Composite foods,One-dish meals,en:one-dish-meals,b,32.000000,0.3,0.1,5.3,3.9,0.9,0.420,0.168
2,000000000114,http://world-fr.openfoodfacts.org/produit/0000...,2021-01-06 15:00:29+00:00,https://images.openfoodfacts.org/images/produc...,Chocolate n 3,80 g,unknown,unknown,None,None,2439.000000,44.0,28.0,30.0,27.0,2.1,0.025,0.010
3,0000000001281,http://world-fr.openfoodfacts.org/produit/0000...,2022-02-11 08:24:48+00:00,https://images.openfoodfacts.org/images/produc...,Tarte noix de coco,None,Sugary snacks,Biscuits and cakes,en:biscuits-and-cakes,d,381.000000,22.0,15.5,27.3,21.9,4.6,0.100,0.040
4,0000000001885,http://world-fr.openfoodfacts.org/produit/0000...,2018-02-08 21:48:11+00:00,https://images.openfoodfacts.org/images/produc...,Compote de poire,None,Fruits and vegetables,Fruits,en:fruits,a,157.000000,0.0,0.0,36.0,27.0,0.6,0.000,0.000


In [4]:
FoodData.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 364286 entries, 0 to 364285
Data columns (total 18 columns):
 #   Column                  Non-Null Count   Dtype              
---  ------                  --------------   -----              
 0   code                    364286 non-null  object             
 1   url                     364286 non-null  object             
 2   last_modified_datetime  364286 non-null  datetime64[ns, UTC]
 3   image_small_url         352291 non-null  object             
 4   product_name            363211 non-null  object             
 5   quantity                219502 non-null  object             
 6   pnns_groups_1           364286 non-null  object             
 7   pnns_groups_2           364286 non-null  object             
 8   food_groups             297058 non-null  object             
 9   nutriscore_grade        315100 non-null  object             
 10  energy-kcal_100g        364156 non-null  float64            
 11  fat_100g                36

## 2.1. Conversion des quantités en nombre

En observant les premières lignes de la base de données, nous nous apercevons que la donnée indiquant la quantité de produit n'est pas saisie sous un format numérique. Nous allons donc créer une fonction pour en extraire les éléments numériques.

In [5]:
def extract_quantities(quantity_string, quantity_max):
    
    '''
    Fonction pour extraire les quantités (poids / volume) de données entrées en string, tenant compte de formats possibles.
    
    Paramètres:
    -----------
    - quantity_string : string contenant les éléments relatifs à une quantité en poids / volume.
    - quantity_max : quantité maximale acceptée comme étant possible
    
    Résultat:
    ---------
    float représentant le poids extrait de quantity_string, ou None si aucune quantité n'a pu être extraite.
    '''
    
    quantity_string = str(quantity_string)
    quantity_string = quantity_string.lower().replace(',','.')
    
    #Pattern identifiant un nombre
    num_pattern = r'\d+\.?\d*'
    #Pattern identifiant une multiplication entre nombres
    mult_pattern = '(\d+\.?\d*\s*g?\s*[\*|x]\s*\d+\.?\d*)'
    
    #Si la description contient une information en (k)g, nous supprimons l'information en (fl) oz
    if re.search(r'g', quantity_string) is not None:
        quantity_string = re.sub(r'\d+\.?\d?\s?(fl)?\.?\s?oz',"", quantity_string)
        
    #Si la description contient une information en L, nous supprimons l'information en (fl) oz et en (k)g
    if re.search(r'[\s|m|c|d\d]l', quantity_string) is not None:
        quantity_string = re.sub(r'\d+\.?\d?\s?(((fl)?\.?\s?oz)|(k?g))',"", quantity_string)
    
    #Selon l'information contenue, nous allons devoir multiplier les valeurs pour obtenir une information homogène en g ou mL
    if re.search(r'ml\b', quantity_string) is not None:
        mult = 1
    elif re.search(r'cl\b', quantity_string) is not None:
        mult = 10
    elif re.search(r'dl\b', quantity_string) is not None:
        mult = 100
    elif re.search(r'fl\.?\s?oz', quantity_string) is not None:
        mult = 29.5735
    elif re.search(r'oz', quantity_string) is not None:
        mult = 28.3495
    elif re.search(r'kg\b|l\b', quantity_string) is not None:
        mult = 1000
    else:
        mult = 1
        
    #Si il n'y a pas de données numériques, alors nous ne pouvons rien extraire
    if re.search(r'\d+', quantity_string) is None:
        return None
    
    else:
        #Si la donnée numérique est de la forme (x produits * y), alors nous devons prendre en compte cette multiplication
        if re.search(mult_pattern, quantity_string) is not None:
            
            mult_values = [float(x) for x in re.findall(num_pattern,re.findall(mult_pattern, quantity_string)[0])]
            result = 1
            for x in mult_values:
                result = result*x
            
            if result <= quantity_max:
                return result
            else:
                return None
        
        else:
            results = [float(x) for x in re.findall(num_pattern, quantity_string)]
            
            if min(results) > quantity_max:
                return None
            
            else:
                results = [x for x in results if x<=quantity_max]
                result = max(results)*mult 
                
                if result <= quantity_max:
                    return result
                else:
                    return None

Nous l'appliquons à notre base de données. Nous définissons la quantité maximale possible sur la base d'un pack de 6 bouteilles de 1,5L.

In [6]:
quantity_max = 6*1500
FoodData.loc[:,'quantity'] = [extract_quantities(x, quantity_max) for x in FoodData["quantity"]]

## 2.2. Valeurs aberrantes

Nous cherchons ensuite à identifier les valeurs nutritionnelles, données numériques, avant de procéder à un premier traitement sur celles-ci.

Nous savons que ces indicateurs sont de la forme `indicateur_100g`.

In [7]:
numeric_fields = FoodData.columns[FoodData.columns.str.contains(r"_100g")].tolist()

In [8]:
display(Markdown(f"Nous retenons {len(numeric_fields)} indicateurs numériques au total."))

Nous retenons 8 indicateurs numériques au total.

Compte tenu des indicateurs retenus, nous retirons les données annonçant plus de 100g d'un élément nutritionnel pour 100g de l'aliment.

In [9]:
numeric_fields_max_100 = numeric_fields.copy()
numeric_fields_max_100.remove('energy-kcal_100g')

FoodData = FoodData.loc[~(FoodData[numeric_fields_max_100]>100).any(axis = 1),:]
FoodData.reset_index(inplace = True, drop = True)

Nous allons également supprimer les lignes pour lesquelles la somme des valeurs nutritionnelles / 100g est supérieure à 100.

In [10]:
first_sum = FoodData.loc[:,["fat_100g", "carbohydrates_100g", "proteins_100g", "salt_100g"]].sum(axis = 1, skipna = True)

# Dans le cas où des valeurs seraient manquantes sur les principaux éléments pris en compte précedemment
second_sum = np.nansum([
    FoodData["saturated-fat_100g"]*FoodData["fat_100g"].isna(), 
    FoodData["sugars_100g"]*FoodData["carbohydrates_100g"].isna(),
    FoodData["sodium_100g"]*FoodData["salt_100g"].isna()
],axis=0)

total_sum = first_sum + second_sum

FoodData = FoodData.loc[total_sum <= 100, :]

Nous allons également plafonner les valeur de certains indicateurs par celles des indicateurs qui les contiennent.

In [11]:
map_capping = FoodData['saturated-fat_100g']>FoodData['fat_100g']
FoodData.loc[map_capping, 'saturated-fat_100g'] = FoodData.loc[map_capping, 'fat_100g']

map_capping = FoodData['sugars_100g']>FoodData['carbohydrates_100g']
FoodData.loc[map_capping, 'sugars_100g'] = FoodData.loc[map_capping, 'carbohydrates_100g']

Nous pouvons également mettre un filtre sur les calories contenues dans les aliments. Une connaissance métier indique qu'il n'existe pas d'aliments apportant plus de 1,000 kcal / 100g. En tenant compte d'une potentielle erreur de conversion dans la saisie, nous pensons pouvoir retirer toutes les données affichant > 4,186.8 kcal / 100g.

Afin de nous en assurer, nous allons étudier les éléments saisis.

In [12]:
kj_to_kcal = 1/4.1868

In [13]:
FoodData.loc[FoodData['energy-kcal_100g']>1000/kj_to_kcal,"url"].values

array(['http://world-fr.openfoodfacts.org/produit/25168242/barres-noisettes-et-chocolat-equitable-bio-la-vie',
       'http://world-fr.openfoodfacts.org/produit/3266191106540/acerola-1000-la-vie-claire',
       'http://world-fr.openfoodfacts.org/produit/37600057990089/bruschetta-papi-mo',
       'http://world-fr.openfoodfacts.org/produit/4056489028642/red-smoothie-solevita',
       'http://world-fr.openfoodfacts.org/produit/9353323000471/deliciou-smoky-bbq-bacon'],
      dtype=object)

Une lecture des éléments saisis montre de claires incohérences entre les données réelles et la saisie. Nous les supprimons donc.

In [14]:
FoodData = FoodData.loc[FoodData['energy-kcal_100g']<=1000/kj_to_kcal,:]

Comme nous l'avons dit, il ne devrait pas y avoir de saisies à plus de 1,000 kcal / 100g. Néanmoins, il est possible que certains éléments aient été saisis en entrant l'information en kJ au lieu de kcal. Nous regardons un échantillon pour nous faire une opinion.

In [15]:
FoodData.loc[FoodData['energy-kcal_100g']>=1000,"url"].sample(5).values

array(['http://world-fr.openfoodfacts.org/produit/5024278001298/9nine-bar-hindbaer-og-chia-med-carob-overtraek',
       'http://world-fr.openfoodfacts.org/produit/7613034453600/cacao-nesquik',
       'http://world-fr.openfoodfacts.org/produit/9300683071559/curry-traditionnel-60-g-keen-s-foods',
       'http://world-fr.openfoodfacts.org/produit/3389090020882/rillettes-sardines-fines-herbes-bio-ty-gwenn',
       'http://world-fr.openfoodfacts.org/produit/8410010260042/aceite-de-oliva-virgen-extra-carbonell'],
      dtype=object)

Une lecture d'un échantillon de données saisies montre que les données en kJ ont été entrées au lieu de celles en kcal. Nous allons donc convertir en kcal tous les éléments saisis avec plus de 1,000 kcal / 100g.

In [16]:
FoodData.loc[FoodData['energy-kcal_100g']>=1000,'energy-kcal_100g'] = FoodData.loc[FoodData['energy-kcal_100g']>=1000,'energy-kcal_100g']*kj_to_kcal

Nous allons également considérer que les 0 entrés dans cet indicateur peuvent en réalité représenter une absence de saisie, pour les aliments qui ont des éléments énergétiques (protéines, graisses, glucides).

In [17]:
mapping = (FoodData['energy-kcal_100g']==0)&((FoodData[['fat_100g', 'saturated-fat_100g', 'carbohydrates_100g', 'sugars_100g', 'proteins_100g']]!=0).any(axis = 1))
FoodData.loc[mapping, 'energy-kcal_100g'] = None

Enfin, nous supprimons toutes les données négatives.

In [18]:
FoodData = FoodData.loc[(FoodData[numeric_fields]>=0).all(axis = 1),:]

In [19]:
FoodData.reset_index(inplace = True, drop = True)

In [20]:
display(Markdown(f"La base de donnée contient désormais {FoodData.shape[0]:,d} lignes et {FoodData.shape[1]:,d} colonnes."))

La base de donnée contient désormais 360,574 lignes et 18 colonnes.

Nous pouvons l'enregistrer, pour l'utiliser dans la phase d'analyse.

In [21]:
FoodData.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 360574 entries, 0 to 360573
Data columns (total 18 columns):
 #   Column                  Non-Null Count   Dtype              
---  ------                  --------------   -----              
 0   code                    360574 non-null  object             
 1   url                     360574 non-null  object             
 2   last_modified_datetime  360574 non-null  datetime64[ns, UTC]
 3   image_small_url         348698 non-null  object             
 4   product_name            359507 non-null  object             
 5   quantity                216515 non-null  float64            
 6   pnns_groups_1           360574 non-null  object             
 7   pnns_groups_2           360574 non-null  object             
 8   food_groups             294259 non-null  object             
 9   nutriscore_grade        312568 non-null  object             
 10  energy-kcal_100g        360574 non-null  float64            
 11  fat_100g                36

In [22]:
FoodData.to_feather('data/FoodData_wrangled.feather')